## Renaming
- IPC522-time-random_num-preset-weather-image_num

In [1]:
import urllib.request
import re

def getWeather(date, stn = "112"):
    year = date[:4]
    mm = date[4:6]
    dd = date[6:]
    # print(year, mm, dd)
    # url = "https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn=" + stn + "&yy=" + year + "&mm=" + mm + "&obs=1"
    url = "https://web.kma.go.kr/weather/climate/past_cal.jsp?stn=" + stn + "&yy=" + year + "&mm=" + mm + "&obs=1&x=24&y=9"
    
# https://www.weather.go.kr/weather/climate/past_cal.jsp?stn=112&yy=2021&mm=07&obs=1&x=24&y=9 ##2107
    lines = []

    f = urllib.request.urlopen(url)
    r = f.read()
    f.close()
    r2 = r.decode('euc-kr', 'ignore')

    lines = r2.split('\n')

    regex = '.*<td class="align_left">평균기온:(.*?)<br \/>최고기온:(.*?)<br \/>최저기온:(.*?)<br \/>평균운량:(.*?)<br \/>일강수량:(.*?)<br \/><\/td>'

    dict_month = {}
    day = 1
    dd = int(dd)
    for l in lines:
        if not '평균기온' in l: continue

        l = l.replace("℃", "")

        l_reg = re.match(regex, l)
        if not l_reg: continue

        dict_day = {'cloud':0, 'rain':0}

        data_cloud = l_reg.groups()[3]   # 평균운량
        data_rain = l_reg.groups()[4]    # 일강수량

        dict_day['cloud'] = data_cloud   # 평균운량
        dict_day['rain'] = data_rain.replace("-", "0").replace("mm", "")    # 일강수량
        if day == dd:
            dict_month[dd] = dict_day
        day = day + 1

    for (day, dict_day) in dict_month.items():
        # print ("{0}{1}{2}, cloud : {3}, rain : {4} ".format(year, mm.zfill(2), str(day).zfill(2), dict_day['cloud'], dict_day['rain']))
        _date = year + mm.zfill(2) + str(day).zfill(2)
        # print(mm)
        # print(day)
        # print(_date)
        _cloud = dict_day['cloud']
        _rain = dict_day['rain']
    return _date, _cloud, _rain

In [2]:
def get_weather(time):
    date, cloud, rain = getWeather(time)
    
    if float(cloud) < 5. and float(rain) == 0.:
            weather = 'sunny'
    elif float(rain) != 0.:
        weather = 'rainy'
    elif float(cloud) > 5. and float(rain) == 0.:
        weather = 'foggy'
            
    return weather

In [3]:
import os
import glob

import pandas as pd
import numpy as np

In [4]:
df = pd.read_excel('rename.xlsx')
df.head()

,before,time,preset,start_point
0,00000000.png,20210511142225,1,1.0
1,00000001.png,20210511142225,1,NaN
2,00000002.png,20210511142225,1,NaN
3,00000003.png,20210511142225,1,NaN
4,00000004.png,20210511142225,1,NaN


In [5]:
df['random_num'] = -1
df['weather'] = -1
df['image_num'] = -1
df['new_filename'] = -1

df = df[['before', 'start_point', 'time', 'random_num', 'preset', 'weather', 'image_num', 'new_filename']]
df.head()

,before,start_point,time,random_num,preset,weather,image_num,new_filename
0,00000000.png,1.0,20210511142225,-1,1,-1,-1,-1
1,00000001.png,NaN,20210511142225,-1,1,-1,-1,-1
2,00000002.png,NaN,20210511142225,-1,1,-1,-1,-1
3,00000003.png,NaN,20210511142225,-1,1,-1,-1,-1
4,00000004.png,NaN,20210511142225,-1,1,-1,-1,-1


In [6]:
id_list = df['time'].unique().tolist()

for i, time in enumerate(id_list):
    data_len = len(df.loc[df['time'] == time])
    
    df.loc[df['time'] == time, 'random_num'] = str(i).zfill(3)
    df.loc[df['time'] == time, 'weather'] = get_weather(str(time)[:8])
    df.loc[df['time'] == time, 'image_num'] = range(data_len)
    df.loc[df['time'] == time, 'image_num'] = df.loc[df['time'] == time, 'image_num'].astype(str).str.zfill(5)
    df.loc[df['time'] == time, 'preset'] = df.loc[df['time'] == time, 'preset'].astype(str).str.zfill(3)
    
df

,before,start_point,time,random_num,preset,weather,image_num,new_filename
0,00000000.png,1.0,20210511142225,000,001,foggy,00000,-1
1,00000001.png,NaN,20210511142225,000,001,foggy,00001,-1
2,00000002.png,NaN,20210511142225,000,001,foggy,00002,-1
3,00000003.png,NaN,20210511142225,000,001,foggy,00003,-1
4,00000004.png,NaN,20210511142225,000,001,foggy,00004,-1
...,...,...,...,...,...,...,...,...
1474,preset_1_20210621_093619.jpeg,NaN,20210621090015,033,001,sunny,00009,-1
1475,preset_1_20210621_094019.jpeg,NaN,20210621090015,033,001,sunny,00010,-1
1476,preset_1_20210621_094419.jpeg,NaN,20210621090015,033,001,sunny,00011,-1
1477,preset_1_20210621_094820.jpeg,NaN,20210621090015,033,001,sunny,00012,-1


In [ ]:
cols = ['time', 'random_num', 'preset', 'weather', 'image_num']
df['new_filename'] = df[cols].apply(lambda x: '_'.join(x.values.astype(str)), axis=1)
df['new_filename'] = 'IPC522_' + df['new_filename'].astype(str) + '.jpg'
df.drop(['start_point', 'random_num', 'weather', 'image_num'], inplace=True, axis=1)

df.to_csv('renamed_filename.csv', index=False)